## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print("Libraries imported successfully!")

## 2. Load and Explore Data

In [ ]:
# Load the station dataset
df = pd.read_csv('station_1068_dataset.csv')
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
# Check for missing values
print("Missing values:")
print(df.isnull().sum())
print(f"\nData types:")
print(df.dtypes)

## 3. Data Preprocessing and Normalization

In [ ]:
# Select features for modeling
feature_columns = ['occupancy', 'e_price', 's_price', 
                   'T', 'P0', 'U', 'nRAIN', 'Td',
                   'hour', 'day_of_week', 'is_weekend']

df_model = df[feature_columns].copy()

print(f"Selected features: {len(feature_columns)}")
print(f"Dataset shape: {df_model.shape}")
print(f"\nSummary statistics:")
print(df_model.describe())

In [ ]:
# Split data into train, validation, and test sets
n = len(df_model)
train_df = df_model[0:int(n*0.7)]
val_df = df_model[int(n*0.7):int(n*0.9)]
test_df = df_model[int(n*0.9):]

print(f"Training set: {len(train_df)} samples ({len(train_df)/n*100:.1f}%)")
print(f"Validation set: {len(val_df)} samples ({len(val_df)/n*100:.1f}%)")
print(f"Test set: {len(test_df)} samples ({len(test_df)/n*100:.1f}%)")
print(f"\nTime ranges:")
print(f"  Train: {train_df.index[0]} to {train_df.index[-1]}")
print(f"  Val: {val_df.index[0]} to {val_df.index[-1]}")
print(f"  Test: {test_df.index[0]} to {test_df.index[-1]}")

In [ ]:
# Normalize the data
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

print("Data normalized using training set statistics")
print(f"\nNormalized training data:")
print(train_df.describe())

## 4. Create WindowGenerator Class

This class handles the creation of time series windows for training and prediction.

In [ ]:
class WindowGenerator:
    def __init__(self, input_width, label_width, shift,
                 train_df, val_df, test_df,
                 label_columns=None):
        # Store the raw data
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)

        ds = ds.map(self.split_window)
        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

print("WindowGenerator class defined")

In [ ]:
# Test the WindowGenerator
# Example: Use 24 hours of data to predict next 12 hours
w1 = WindowGenerator(input_width=24, label_width=12, shift=12,
                     train_df=train_df, val_df=val_df, test_df=test_df,
                     label_columns=['occupancy'])

print(w1)
print(f"\nExample batch shapes:")
example_inputs, example_labels = w1.example
print(f"  Inputs shape: {example_inputs.shape}")
print(f"  Labels shape: {example_labels.shape}")

## 5. Create Plotting Function

In [ ]:
def plot_predictions(window, model, plot_col='occupancy', max_subplots=3):
    """Plot model predictions vs actual values"""
    inputs, labels = window.example
    plt.figure(figsize=(16, 8))
    plot_col_index = window.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    
    for n in range(max_n):
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col} [normed]')
        plt.plot(window.input_indices, inputs[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=-10)

        if window.label_columns:
            label_col_index = window.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(window.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        
        if model is not None:
            predictions = model(inputs)
            plt.scatter(window.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)

        if n == 0:
            plt.legend()

    plt.xlabel('Time [h]')
    plt.tight_layout()
    plt.show()

print("Plotting function defined")

## 6. Build Baseline Models

In [ ]:
# Baseline 1: Last value (persistence)
class Baseline(tf.keras.Model):
    def __init__(self, label_index=None):
        super().__init__()
        self.label_index = label_index

    def call(self, inputs):
        if self.label_index is None:
            return inputs
        # Take the last value from input and repeat it for the prediction window
        result = inputs[:, -1:, self.label_index]
        return tf.tile(result[:, :, tf.newaxis], [1, 12, 1])

# Create and test baseline
baseline = Baseline(label_index=w1.column_indices['occupancy'])

baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                 metrics=[tf.keras.metrics.MeanAbsoluteError()])

val_performance = {}
performance = {}

val_performance['Baseline'] = baseline.evaluate(w1.val, verbose=0)
performance['Baseline'] = baseline.evaluate(w1.test, verbose=0)

print("Baseline Model (Last Value Persistence):")
print(f"  Validation - Loss: {val_performance['Baseline'][0]:.4f}, MAE: {val_performance['Baseline'][1]:.4f}")
print(f"  Test - Loss: {performance['Baseline'][0]:.4f}, MAE: {performance['Baseline'][1]:.4f}")

In [ ]:
# Visualize baseline predictions
plot_predictions(w1, baseline, plot_col='occupancy', max_subplots=3)

## 7. Build Dense (Fully Connected) Model

In [ ]:
# Dense model with multiple layers
# Flatten input, pass through dense layers, reshape to match label_width
dense_model = tf.keras.Sequential([
    layers.Flatten(),
    layers.Dense(units=64, activation='relu'),
    layers.Dense(units=64, activation='relu'),
    layers.Dense(units=w1.label_width),  # Output for 12 time steps
    layers.Reshape([w1.label_width, 1])  # Reshape to (batch, 12, 1)
])

# Compile the model
dense_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=[tf.keras.metrics.MeanAbsoluteError()])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=30,
    mode='min',
    restore_best_weights=True)

print("Dense model created")
print(dense_model.summary())

In [ ]:
# Train dense model
history_dense = dense_model.fit(
    w1.train,
    epochs=50,
    validation_data=w1.val,
    callbacks=[early_stopping],
    verbose=1)

# Evaluate
val_performance['Dense'] = dense_model.evaluate(w1.val, verbose=0)
performance['Dense'] = dense_model.evaluate(w1.test, verbose=0)

print(f"\nDense Model:")
print(f"  Validation - Loss: {val_performance['Dense'][0]:.4f}, MAE: {val_performance['Dense'][1]:.4f}")
print(f"  Test - Loss: {performance['Dense'][0]:.4f}, MAE: {performance['Dense'][1]:.4f}")

In [ ]:
# Get the element specification of the dataset
print("Training dataset element spec:")
print(w1.train.element_spec)

# Or get shapes from a batch
inputs, labels = next(iter(w1.train))
print(f"\nBatch shapes:")
print(f"  Inputs: {inputs.shape}")
print(f"  Labels: {labels.shape}")

In [ ]:
print(dense_model.summary())

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_dense.history['loss'], label='Training Loss')
plt.plot(history_dense.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Dense Model Training History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_dense.history['mean_absolute_error'], label='Training MAE')
plt.plot(history_dense.history['val_mean_absolute_error'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Dense Model MAE History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize dense model predictions
plot_predictions(w1, dense_model, plot_col='occupancy', max_subplots=3)

## 8. Build LSTM Model

In [ ]:
# LSTM model - outputs only the last 12 timesteps
lstm_model = tf.keras.Sequential([
    layers.LSTM(32, return_sequences=True),
    layers.Dense(units=1),
    layers.Lambda(lambda x: x[:, -w1.label_width:, :])  # Take only last 12 timesteps
])

lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                   optimizer=tf.keras.optimizers.Adam(),
                   metrics=[tf.keras.metrics.MeanAbsoluteError()])

print("LSTM model created")
print(lstm_model.summary())

In [ ]:
# Train LSTM model
history_lstm = lstm_model.fit(
    w1.train,
    epochs=50,
    validation_data=w1.val,
    callbacks=[early_stopping],
    verbose=1)

# Evaluate
val_performance['LSTM'] = lstm_model.evaluate(w1.val, verbose=0)
performance['LSTM'] = lstm_model.evaluate(w1.test, verbose=0)

print(f"\nLSTM Model:")
print(f"  Validation - Loss: {val_performance['LSTM'][0]:.4f}, MAE: {val_performance['LSTM'][1]:.4f}")
print(f"  Test - Loss: {performance['LSTM'][0]:.4f}, MAE: {performance['LSTM'][1]:.4f}")

In [ ]:
# Plot LSTM training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_lstm.history['loss'], label='Training Loss')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('LSTM Model Training History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_lstm.history['mean_absolute_error'], label='Training MAE')
plt.plot(history_lstm.history['val_mean_absolute_error'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('LSTM Model MAE History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize LSTM predictions
plot_predictions(w1, lstm_model, plot_col='occupancy', max_subplots=3)

## 9. Advanced Multi-Step Model Configuration

In [ ]:
# Create windows with different configurations
# Multi-output: predict next 24 hours using last 24 hours
multi_window = WindowGenerator(input_width=24, label_width=24, shift=24,
                                train_df=train_df, val_df=val_df, test_df=test_df,
                                label_columns=['occupancy'])

print("Multi-step Window Configuration:")
print(multi_window)

# Build multi-step LSTM
multi_lstm_model = tf.keras.Sequential([
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(32, return_sequences=True),
    layers.Dense(units=1)
])

multi_lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(),
                          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                          metrics=[tf.keras.metrics.MeanAbsoluteError()])

print("\nMulti-step LSTM model created")
print(multi_lstm_model.summary())

In [ ]:
# Train multi-step LSTM
history_multi = multi_lstm_model.fit(
    multi_window.train,
    epochs=50,
    validation_data=multi_window.val,
    callbacks=[early_stopping],
    verbose=1)

# Evaluate
val_performance['Multi-LSTM'] = multi_lstm_model.evaluate(multi_window.val, verbose=0)
performance['Multi-LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)

print(f"\nMulti-step LSTM Model:")
print(f"  Validation - Loss: {val_performance['Multi-LSTM'][0]:.4f}, MAE: {val_performance['Multi-LSTM'][1]:.4f}")
print(f"  Test - Loss: {performance['Multi-LSTM'][0]:.4f}, MAE: {performance['Multi-LSTM'][1]:.4f}")

In [ ]:
# Plot multi-step LSTM training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_multi.history['loss'], label='Training Loss')
plt.plot(history_multi.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Multi-step LSTM Training History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_multi.history['mean_absolute_error'], label='Training MAE')
plt.plot(history_multi.history['val_mean_absolute_error'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Multi-step LSTM MAE History')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize multi-step predictions
plot_predictions(multi_window, multi_lstm_model, plot_col='occupancy', max_subplots=3)

## 10. Model Comparison

In [ ]:
# Compare all models
x = np.arange(len(performance))
width = 0.3

fig, ax = plt.subplots(figsize=(12, 6))

# Plot test performance
test_mae = [performance[name][1] for name in performance]
test_loss = [performance[name][0] for name in performance]

ax.bar(x - width/2, test_mae, width, label='MAE', color='steelblue', alpha=0.8)
ax.bar(x + width/2, test_loss, width, label='MSE Loss', color='coral', alpha=0.8)

ax.set_ylabel('Error')
ax.set_title('Model Performance Comparison (Test Set)')
ax.set_xticks(x)
ax.set_xticklabels(performance.keys(), rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nTest Set Performance Summary:")
print("=" * 60)
for name in performance:
    print(f"{name:15s} - MSE: {performance[name][0]:.4f}, MAE: {performance[name][1]:.4f}")
print("=" * 60)

## 11. Detailed Predictions on Test Set

In [ ]:
# Generate predictions on entire test set for multi-step model
def predict_on_test_set(model, window, test_df, train_mean, train_std):
    """Generate predictions for entire test set and denormalize"""
    predictions = []
    actuals = []
    
    # Get test dataset without shuffling for sequential prediction
    test_data = np.array(window.test_df, dtype=np.float32)
    test_ds = tf.keras.utils.timeseries_dataset_from_array(
        data=test_data,
        targets=None,
        sequence_length=window.total_window_size,
        sequence_stride=1,
        shuffle=False,
        batch_size=32)
    
    test_ds = test_ds.map(window.split_window)
    
    for inputs, labels in test_ds:
        preds = model(inputs)
        predictions.append(preds.numpy())
        actuals.append(labels.numpy())
    
    # Concatenate all batches
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    
    # Denormalize occupancy predictions
    occ_col_idx = window.column_indices['occupancy']
    predictions_denorm = predictions * train_std['occupancy'] + train_mean['occupancy']
    actuals_denorm = actuals * train_std['occupancy'] + train_mean['occupancy']
    
    return predictions_denorm, actuals_denorm

# Get predictions
preds_multi, actuals_multi = predict_on_test_set(multi_lstm_model, multi_window, 
                                                  test_df, train_mean, train_std)

print(f"Predictions shape: {preds_multi.shape}")
print(f"Actuals shape: {actuals_multi.shape}")
print(f"\nDenormalized predictions range: [{preds_multi.min():.2f}, {preds_multi.max():.2f}]")
print(f"Denormalized actuals range: [{actuals_multi.min():.2f}, {actuals_multi.max():.2f}]")

In [ ]:
# Plot predictions vs actuals for first few samples
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

for i, ax in enumerate(axes):
    # Plot actual values
    ax.plot(range(24), actuals_multi[i, :, 0], 
           label='Actual', linewidth=2, marker='o', alpha=0.7, color='steelblue')
    # Plot predictions
    ax.plot(range(24), preds_multi[i, :, 0], 
           label='Predicted', linewidth=2, marker='s', alpha=0.7, color='coral', linestyle='--')
    
    ax.set_xlabel('Hours Ahead')
    ax.set_ylabel('Occupancy')
    ax.set_title(f'Sample {i+1}: 24-Hour Multi-Step Forecast')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Aggregate metrics by forecast horizon
from sklearn.metrics import mean_absolute_error, mean_squared_error

horizons = range(1, 25)
mae_by_horizon = []
rmse_by_horizon = []

for h in horizons:
    h_idx = h - 1
    mae = mean_absolute_error(actuals_multi[:, h_idx, 0], preds_multi[:, h_idx, 0])
    rmse = np.sqrt(mean_squared_error(actuals_multi[:, h_idx, 0], preds_multi[:, h_idx, 0]))
    mae_by_horizon.append(mae)
    rmse_by_horizon.append(rmse)

# Plot error by horizon
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].plot(horizons, mae_by_horizon, marker='o', linewidth=2, color='steelblue')
axes[0].set_xlabel('Forecast Horizon (hours)')
axes[0].set_ylabel('MAE')
axes[0].set_title('Mean Absolute Error by Forecast Horizon')
axes[0].grid(True, alpha=0.3)

axes[1].plot(horizons, rmse_by_horizon, marker='s', linewidth=2, color='coral')
axes[1].set_xlabel('Forecast Horizon (hours)')
axes[1].set_ylabel('RMSE')
axes[1].set_title('Root Mean Squared Error by Forecast Horizon')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Forecast Performance by Horizon:")
print(f"  1-hour ahead:  MAE={mae_by_horizon[0]:.4f}, RMSE={rmse_by_horizon[0]:.4f}")
print(f"  6-hour ahead:  MAE={mae_by_horizon[5]:.4f}, RMSE={rmse_by_horizon[5]:.4f}")
print(f"  12-hour ahead: MAE={mae_by_horizon[11]:.4f}, RMSE={rmse_by_horizon[11]:.4f}")
print(f"  24-hour ahead: MAE={mae_by_horizon[23]:.4f}, RMSE={rmse_by_horizon[23]:.4f}")

## 12. Summary

In [ ]:
print("=" * 70)
print("MULTI-STEP FORECASTING - FINAL SUMMARY")
print("=" * 70)
print(f"\nDataset:")
print(f"  Station: 1068 (271 charging piles)")
print(f"  Total samples: {len(df)}")
print(f"  Features: {len(feature_columns)}")
print(f"  Train/Val/Test split: 70%/20%/10%")

print(f"\nModels Evaluated:")
for name in performance:
    print(f"  {name:15s} - Test MSE: {performance[name][0]:.4f}, Test MAE: {performance[name][1]:.4f}")

print(f"\nBest Model: {min(performance, key=lambda x: performance[x][1])}")
print(f"  (Based on lowest MAE)")

print(f"\nMulti-step LSTM Configuration:")
print(f"  Input window: 24 hours")
print(f"  Output window: 24 hours")
print(f"  Architecture: 2 LSTM layers (64, 32 units) + Dense output")
print(f"  Dropout: 0.2")

print("\n" + "=" * 70)
print("The multi-step LSTM model predicts 24 hours ahead using 24 hours of")
print("historical data, including occupancy, prices, weather, and temporal features.")
print("Error increases with forecast horizon, as expected for time series forecasting.")
print("=" * 70)